# Optimización de matching de portadas


 ## Importación de librerías e inicialización
 Modificar los paths a la imagen de query y a la carpeta del conjunto de imagenes en el que buscar

In [0]:
import os
import cv2
from matplotlib import pyplot as plt

min_number_of_matches = 300

main_image_path = "drive/My Drive/Universidad/Master/Herramientas software/main.jpg"
covers_folder = "drive/My Drive/Universidad/Master/Herramientas software/covers"

## Creación de matcher ORB

In [0]:
orb = cv2.ORB_create(nfeatures=2000)
main_image = cv2.imread(main_image_path, cv2.IMREAD_GRAYSCALE)
main_image_kps, main_image_des = orb.detectAndCompute(main_image, None)

images_paths = os.listdir(covers_folder)
images_paths.sort()

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

max_num_of_matches = 0
max_matches_img = None
max_matches_img_kps = None
max_matches_img_matches = None

## Ejecucion sin paralelización

In [9]:
import time
start_time = time.time()

for image_path in images_paths:
    image = cv2.imread(covers_folder + "/" + image_path, cv2.IMREAD_GRAYSCALE)
    image_kps, image_des = orb.detectAndCompute(image, None)
    matches = bf.match(main_image_des, image_des)
    num_of_matches = len(matches)

    if num_of_matches > max_num_of_matches:
        max_num_of_matches = num_of_matches
        max_matches_img = image
        max_matches_img_kps = image_kps
        max_matches_img_matches = matches

print("--- %s seconds ---" % (time.time() - start_time))

--- 4.811753511428833 seconds ---


Sin realizar optimizaciones para la paralelización con el entorno virtual de Google colab se obtienen tiempos de alrededor de 4,7 segundos.

Un tiempo considerablemente rapido ya que elimina gran parte de los accesos a memoria

## Busqueda de número de matches con paralelización

In [8]:
import time
import numpy as np
from multiprocessing import Pool
from functools import partial

def get_matches(index, main_image_des):
  image_path = images_paths[index]
  image = cv2.imread(covers_folder + "/" + image_path, cv2.IMREAD_GRAYSCALE)
  image_kps, image_des = orb.detectAndCompute(image, None)
  matches = bf.match(main_image_des, image_des)
  num_of_matches = len(matches)
  return num_of_matches

p = Pool(len(images_paths))

start_time = time.time()
prod_x=partial(get_matches, main_image_des=main_image_des)
res = p.map_async(prod_x, np.arange(len(images_paths))).get()

# Posterirmente buscaramos el máximo y comprobariamos que es mayor que el umbral 
# minimo y tendriamos que volver a calcular el descriptor para poder pintar los resultados

print("--- %s seconds ---" % (time.time() - start_time))

--- 4.6156511306762695 seconds ---


Debido a como estaba realizado el algoritmo de busqueda no se puede reportar una mejora en el rendimiento, ya que el algoritmo no paralelizado descartaba gran cantidad de información evitando almacenarla si no es una mejor aproximación.

Evitar todas estas escrituras en memoria en el metodo no parelelizado hace bastante más complejo conseguir un resultado más rapido con la paralelización

En el codigo de arriba se muestra que si se puede conseguir una mejora para la obtención de un vector con el numero de matches (alrededor de 0.3 segundos mas rapido en este caso), sin embargo, en este codigo aun faltaría obtener el más alto y asegurarnos de que cumple el umbral minimo, lo que quiere decir que debido a que la primera implementación, no paralelizada, está bastante optimizada en cantidad de escrituras en memoria requeriria de un codigo muy complejo para poder optimizar los tiempos.





## Resultados de la ejecución
Estos resultados se pueden mostrar unicamente tras la ejecucion del bloque sin paralelización ya que, como se comenta en el texto superior el segundo método reporta una ralentización por como se encuentra implementado el algoritmo.

In [0]:
if max_num_of_matches >= min_number_of_matches:
    img3 = None
    img3 = cv2.drawMatches(main_image, main_image_kps, max_matches_img, max_matches_img_kps,
                           max_matches_img_matches, img3, flags=2)
    plt.imshow(img3), plt.show()
else:
    print("Any cover has enough matches with the query image")